In [3]:
import pandas as pd

In [4]:
train = pd.read_csv('./Corona.csv')
test = pd.read_csv('./Corona_NLP_test.csv')

In [5]:
train.Sentiment.unique()

array(['Neutral', 'Positive', 'Extremely Negative', 'Negative',
       'Extremely Positive'], dtype=object)

In [6]:
test.Sentiment.unique()

array(['Extremely Negative', 'Positive', 'Extremely Positive', 'Negative',
       'Neutral'], dtype=object)

In [7]:
train.sample(frac=1).head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
16186,19985,64937,Abu Dhabi,22-03-2020,The #supplyshock instigated by the #COVID-19 o...,Extremely Negative
38793,42592,87544,Bangalore,12-04-2020,How #CoronaVirus has transformed #USA consumer...,Neutral
40043,43842,88794,"San Diego, CA",13-04-2020,Food is the most essential human need. CEO Vin...,Negative
1670,5469,50421,Dublin City Centre,17-03-2020,Give the perfect gift this Mother's Day from o...,Extremely Positive
23511,27310,72262,Canada,26-03-2020,Remember to say thank you to all those working...,Extremely Positive


In [8]:
del train['TweetAt']

In [9]:
del train['UserName']

In [10]:
del train['ScreenName']

In [11]:
del train['Location']

In [12]:
train.sample(frac=1).head()

,OriginalTweet,Sentiment
8455,all grocery store workers should get free test...,Positive
3370,Not a bad little post here from @chow on How t...,Extremely Positive
5237,Amazon stock prices are going to CRASH so bad ...,Extremely Negative
37475,We need to monitor local food prices to unders...,Negative
11702,People need to stop hoarding supplies during t...,Negative


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(train['OriginalTweet'])
test_vectors = vectorizer.transform(test['OriginalTweet'])

In [15]:
import time
from sklearn import svm
from sklearn.metrics import classification_report

In [ ]:
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, train['Sentiment'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1


print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(test['Sentiment'], prediction_linear, output_dict=True)


print('positive: ', report['Positive'])
print('negative: ', report['Negative'])
print('Extremely positive: ', report['Extremely Positive'])
print('Extremely Negative: ', report['Extremely Negative'])
print('Neutral : ', report['Neutral'])